In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv, find_dotenv

In [2]:
_: bool = load_dotenv(find_dotenv())

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")


In [3]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [4]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [5]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [6]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()


In [10]:
intial_state = {'title': 'Climate change is Real'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'Climate change is Real', 'outline': '## Blog Outline: Climate Change is Real: Understanding the Science and Addressing the Challenge\n\n**I. Introduction (Hook the Reader - Establish Urgency & Relevance)**\n\n*   **A. Opening Hook:**\n    *   Start with a compelling statistic about a recent climate-related event (e.g., record heatwave, devastating flood, unprecedented wildfire).\n    *   Share a personal anecdote about witnessing the effects of climate change firsthand.\n    *   Pose a thought-provoking question about the future of our planet.\n*   **B. Briefly Define Climate Change:**\n    *   Distinguish between climate and weather.\n    *   Explain that climate change refers to long-term shifts in temperatures and weather patterns.\n*   **C. Thesis Statement:**\n    *   Clearly state the blog\'s central argument: Climate change is a real and pressing issue, supported by overwhelming scientific evidence, and requires immediate and comprehensive action.\n*   **D. Blog Overv

In [11]:
print(final_state['outline'])

## Blog Outline: Climate Change is Real: Understanding the Science and Addressing the Challenge

**I. Introduction (Hook the Reader - Establish Urgency & Relevance)**

*   **A. Opening Hook:**
    *   Start with a compelling statistic about a recent climate-related event (e.g., record heatwave, devastating flood, unprecedented wildfire).
    *   Share a personal anecdote about witnessing the effects of climate change firsthand.
    *   Pose a thought-provoking question about the future of our planet.
*   **B. Briefly Define Climate Change:**
    *   Distinguish between climate and weather.
    *   Explain that climate change refers to long-term shifts in temperatures and weather patterns.
*   **C. Thesis Statement:**
    *   Clearly state the blog's central argument: Climate change is a real and pressing issue, supported by overwhelming scientific evidence, and requires immediate and comprehensive action.
*   **D. Blog Overview:**
    *   Briefly outline the main points that will be di

In [9]:
print(final_state['content'])

## Rise of AI in India: A Technological Revolution Unfolding

**I. Introduction: Setting the Stage**

Imagine a world where doctors can diagnose diseases with unparalleled accuracy, farmers can optimize crop yields with pinpoint precision, and financial institutions can detect fraud before it even happens. This isn't science fiction; it's the reality unfolding in India, fueled by the rapid rise of Artificial Intelligence (AI). In fact, the Indian AI market is projected to reach a staggering \$15.7 billion by 2026, showcasing its immense potential.

But what exactly is AI? In simple terms, it's the ability of machines to mimic human intelligence – to learn, reason, and solve problems. This blog delves into the heart of the AI revolution in India, exploring the key drivers fueling its growth, the diverse applications transforming various sectors, the challenges that lie ahead, and the exciting potential for the future. We'll cover government initiatives, ethical considerations, and the o